# Azure AI Agenti s podporo za Model Context Protocol (MCP)

Ta zvezek prikazuje, kako uporabiti Azure AI agente z orodji Model Context Protocol (MCP) za ustvarjanje inteligentnega agenta, ki lahko izkorišča zunanje MCP strežnike za izboljšane zmogljivosti.


## Namestitev potrebnih paketov NuGet

Najprej moramo namestiti paket Azure AI Agents Persistent, ki zagotavlja osnovno funkcionalnost za delo z Azure AI Agents.


## Prednosti avtentikacije brez ključev

Ta zvezek prikazuje **avtentikacijo brez ključev**, ki ponuja več prednosti:
- ✅ **Brez upravljanja API ključev** - Uporablja avtentikacijo na podlagi identitete Azure
- ✅ **Izboljšana varnost** - Brez shranjevanja skrivnosti v kodi ali konfiguraciji
- ✅ **Samodejna rotacija poverilnic** - Azure upravlja življenjski cikel poverilnic
- ✅ **Dostop na podlagi vlog** - Uporablja Azure RBAC za natančno določene pravice dostopa

`DefaultAzureCredential` bo samodejno uporabil najboljši razpoložljivi vir poverilnic:
1. Upravljana identiteta (pri izvajanju v Azure)
2. Poverilnice Azure CLI (med razvojem)
3. Poverilnice Visual Studio
4. Spremenljivke okolja (če so konfigurirane)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Namestite paket Azure Identity za preverjanje pristnosti z Azure storitvami z uporabo DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Uvoz potrebnih imen prostorov

Uvozite potrebne imen prostore za Azure AI Agents in Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Konfiguracija Azure AI Agent Client (Avtentikacija brez ključa)

Nastavite konfiguracijske spremenljivke in ustvarite PersistentAgentsClient z uporabo **avtentikacije brez ključa**:
- **projectEndpoint**: Končna točka projekta Azure AI Foundry
- **modelDeploymentName**: Ime nameščenega AI modela (GPT-4.1 nano)
- **mcpServerUrl**: URL strežnika MCP (Microsoft Learn API)
- **mcpServerLabel**: Oznaka za identifikacijo strežnika MCP
- **DefaultAzureCredential**: Uporablja upravljano identiteto, Azure CLI ali druge vire poverilnic (brez potrebnih API ključev)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Ustvarite definicijo orodja MCP

Ustvarite definicijo orodja MCP, ki se poveže z Microsoft Learn MCP strežnikom. To bo agentu omogočilo dostop do vsebine in dokumentacije Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Ustvarite AI agenta

Ustvarite trajnega AI agenta z določenim modelom in orodji MCP. Agent je konfiguriran z:
- Modelom GPT-4.1 nano
- Navodili za uporabo orodij MCP za pomoč
- Dostopom do Microsoft Learn MCP strežnika


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Ustvarite nit pogovora in pošljite sporočilo

Ustvarite nit pogovora in pošljite uporabniško sporočilo z vprašanjem o razliki med Azure OpenAI in OpenAI. To bo preverilo agentovo sposobnost uporabe orodij MCP za zagotavljanje natančnih informacij.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Konfigurirajte vire orodja MCP (brez ključa)

Nastavite vire orodja MCP. Za resnično pristop brez ključa lahko odstranite prilagojene glave, če strežnik MCP podpira preverjanje pristnosti na podlagi identitete Azure. Spodnji primer prikazuje, kako dodati glave, če so potrebne, vendar za scenarije brez ključa to morda ni potrebno.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Začetek izvajanja agenta

Ustvarite in začnite izvajanje za obdelavo uporabnikovega sporočila. Agent bo uporabil konfigurirana MCP orodja in vire za ustvarjanje odgovora.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Spremljanje izvajanja in obravnava odobritev orodij (brez ključa)

Spremljajte stanje izvajanja agenta in obravnavajte morebitne potrebne odobritve orodij. Ta zanka:
1. Čaka, da se izvajanje zaključi ali zahteva ukrepanje
2. Samodejno odobri klice orodij MCP, kadar je to potrebno
3. Pri avtentikaciji brez ključa glave morda niso potrebne, če strežnik MCP podpira identiteto Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Prikaz rezultatov pogovora

Pridobite in prikažite vsa sporočila v niti, ki prikazujejo tako uporabnikovo vprašanje kot agentov odgovor. Sporočila so prikazana v kronološkem vrstnem redu z časovnimi oznakami in indikatorji vlog.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za prevajanje z umetno inteligenco [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem maternem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo profesionalni človeški prevod. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
